<a href="https://colab.research.google.com/github/AreebAhmad-02/Real-time-rag-pipeline-for-rag-research-papers/blob/main/Copy_of_Rag_papers_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install Required Libraries
!pip install arxiv -q -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


# Getting Data Using Arxiv API for Past 2 Years of RAG Pipeline

In [ ]:
! pip install -q feedparser

In [ ]:
"""
python_arXiv_parsing_example.py

This sample script illustrates a basic arXiv api call
followed by parsing of the results using the
feedparser python module.

Please see the documentation at
http://export.arxiv.org/api_help/docs/user-manual.html
for more information, or email the arXiv api
mailing list at arxiv-api@googlegroups.com.

urllib is included in the standard python library.
feedparser can be downloaded from http://feedparser.org/ .

Author: Julius B. Lucks

This is free software.  Feel free to do what you want
with it, but please play nice with the arXiv API!
"""

import urllib.request
import feedparser

# Base api query url
base_url = 'http://export.arxiv.org/api/query?'

# Search parameters
search_query = 'all:rag'  # search for rag in all fields
start = 0                      # retrieve the first 5 results
max_results = 516

query = 'search_query=%s&start=%i&max_results=%i' % (search_query, start, max_results)
# query = 'search_query=%s&start=%i' % (search_query, start)

# Opensearch metadata such as totalResults, startIndex,
# and itemsPerPage live in the opensearch namespace.
# Some entry metadata lives in the arXiv namespace.
# This is a hack to expose both of these namespaces in
# feedparser v4.1
# feedparser.FeedParserMixin.namespaces['http://a9.com/-/spec/opensearch/1.1/'] = 'opensearch'
# feedparser.FeedParserMixin.namespaces['http://arxiv.org/schemas/atom'] = 'arxiv'

pdf_links = {}
# perform a GET request using the base_url and query
response = urllib.request.urlopen(base_url + query).read()

# parse the response using feedparser
feed = feedparser.parse(response)

# print out feed information
print('Feed title: %s' % feed.feed.title)
print('Feed last updated: %s' % feed.feed.updated)

# print opensearch metadata
print('totalResults for this query: %s' % feed.feed.opensearch_totalresults)
print('itemsPerPage for this query: %s' % feed.feed.opensearch_itemsperpage)
print('startIndex for this query: %s' % feed.feed.opensearch_startindex)

# Run through each entry, and print out information
for entry in feed.entries:
    published_year = int(entry.published[0:4])
    if published_year >=2022:
      inner_dict = {}
      print('e-print metadata')
      print('arxiv-id: %s' % entry.id.split('/abs/')[-1])
      print('published date type',type(entry.published))
      print('Published: %s' % entry.published)
      print('Title:  %s' % entry.title)

      inner_dict['Published Date'] = entry.published
      # feedparser v4.1 only grabs the first author
      author_string = entry.author

      # grab the affiliation in <arxiv:affiliation> if present
      # - this will only grab the first affiliation encountered
      #   (the first affiliation for the first author)
      # Please email the list with a way to get all of this information!
      try:
          author_string += ' (%s)' % entry.arxiv_affiliation
      except AttributeError:
          pass

      print('Last Author:  %s' % author_string)

      # feedparser v5.0.1 correctly handles multiple authors, print them all
      try:
          print('Authors:  %s' % ', '.join(author.name for author in entry.authors))
      except AttributeError:
          pass

      # get the links to the abs page and pdf for this e-print

      for link in entry.links:
          count = 1
          if link.rel == 'alternate':
              print('abs page link: %s' % link.href)
              inner_dict['alternate link'+str(count)] = link.href
          elif link.title == 'pdf':
              print('pdf link: %s' % link.href)
              inner_dict['pdf link'+str(count)] =  link.href
          count+=1

      # The journal reference, comments and primary_category sections live under
      # the arxiv namespace
      try:
          journal_ref = entry.arxiv_journal_ref
      except AttributeError:
          journal_ref = 'No journal ref found'
      print('Journal reference: %s' % journal_ref)

      try:
          comment = entry.arxiv_comment
      except AttributeError:
          comment = 'No comment found'
      print('Comments: %s' % comment)

      # Since the <arxiv:primary_category> element has no data, only
      # attributes, feedparser does not store anything inside
      # entry.arxiv_primary_category
      # This is a dirty hack to get the primary_category, just take the
      # first element in entry.tags.  If anyone knows a better way to do
      # this, please email the list!
      print('Primary Category: %s' % entry.tags[0]['term'])

      # Lets get all the categories
      all_categories = [t['term'] for t in entry.tags]
      print('All Categories: %s' % ', '.join(all_categories))

      # The abstract is in the <summary> element
      print('Abstract: %s' % entry.summary)
      pdf_links[entry.title] = inner_dict



Streaming output truncated to the last 5000 lines.
abs page link: http://arxiv.org/abs/2401.10286v3
pdf link: http://arxiv.org/pdf/2401.10286v3
Journal reference: No journal ref found
Comments: No comment found
Primary Category: cs.CL
All Categories: cs.CL, cs.AI
Abstract: In previous studies, code-based models have consistently outperformed
text-based models in reasoning-intensive scenarios. When generating our
knowledge base for Retrieval-Augmented Generation (RAG), we observed that
code-based models also perform exceptionally well in Chinese QA Pair Extraction
task. Further, our experiments and the metrics we designed discovered that
code-based models containing a certain amount of Chinese data achieve even
better performance. Additionally, the capabilities of code-based English models
in specified Chinese tasks offer a distinct perspective for discussion on the
philosophical "Chinese Room" thought experiment.
e-print metadata
arxiv-id: 2402.04206v1
published date type <class 'str'>

In [ ]:
feed.feed.opensearch_totalresults

'516'

In [ ]:
len(pdf_links)


467

In [ ]:
pdf_pubDates = [pdf_links]

In [ ]:
pdf_pubDates

[{'MultiHop-RAG: Benchmarking Retrieval-Augmented Generation for Multi-Hop\n  Queries': {'Published Date': '2024-01-27T11:41:48Z',
   'alternate link1': 'http://arxiv.org/abs/2401.15391v1',
   'pdf link1': 'http://arxiv.org/pdf/2401.15391v1'},
  'Seven Failure Points When Engineering a Retrieval Augmented Generation\n  System': {'Published Date': '2024-01-11T12:04:11Z',
   'alternate link1': 'http://arxiv.org/abs/2401.05856v1',
   'pdf link1': 'http://arxiv.org/pdf/2401.05856v1'},
  'RAGGED: Towards Informed Design of Retrieval Augmented Generation\n  Systems': {'Published Date': '2024-03-14T02:26:31Z',
   'alternate link1': 'http://arxiv.org/abs/2403.09040v1',
   'pdf link1': 'http://arxiv.org/pdf/2403.09040v1'},
  'Observations on Building RAG Systems for Technical Documents': {'Published Date': '2024-03-31T12:01:34Z',
   'alternate link1': 'http://arxiv.org/abs/2404.00657v1',
   'pdf link1': 'http://arxiv.org/pdf/2404.00657v1'},
  'Prompt-RAG: Pioneering Vector Embedding-Free Retrie

In [ ]:
import json
with open('pdfLinks_published_dates.json', 'w') as f:
    json.dump(pdf_links, f, indent=4)

### The below code first takes the url from your json file then downloads the paper in colab local storage and then copies it to your g drive but you will want to upload the json file  with names and links of rag papers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p "/content/drive/My Drive/rag papers"

In [ ]:


import urllib.request

def download_and_save_to_drive(url, filename, drive_path='/content/drive/My Drive/rag papers/'):
    """Downloads a file from the given URL and saves it to Google Drive."""
    local_filename = '/content/' + filename  # Save temporarily in Colab
    urllib.request.urlretrieve(url, local_filename)

    drive_filename = drive_path + filename  # Full path in Google Drive
    !cp "{local_filename}" "{drive_filename}"  # Copy to Google Drive

# Example usage:
paper_url = 'http://arxiv.org/pdf/2404.00657v1'  # Replace with the actual URL
filename = 'Observations on Building RAG Systems for Technical Documents.pdf'  # Choose a filename
download_and_save_to_drive(paper_url, filename)

In [ ]:
with open('/content/pdfLinks_published_dates.json', 'r') as f:
        rag_paper_data = json.load(f)

In [ ]:
for paper_name,paper_data in rag_paper_data.items():
  filename = paper_name
  paper_url = paper_data['pdf link1']
  print(filename)
  print(paper_url)
  download_and_save_to_drive(paper_url, filename)

MultiHop-RAG: Benchmarking Retrieval-Augmented Generation for Multi-Hop
  Queries
http://arxiv.org/pdf/2401.15391v1
Seven Failure Points When Engineering a Retrieval Augmented Generation
  System
http://arxiv.org/pdf/2401.05856v1
RAGGED: Towards Informed Design of Retrieval Augmented Generation
  Systems
http://arxiv.org/pdf/2403.09040v1
Observations on Building RAG Systems for Technical Documents
http://arxiv.org/pdf/2404.00657v1
Prompt-RAG: Pioneering Vector Embedding-Free Retrieval-Augmented
  Generation in Niche Domains, Exemplified by Korean Medicine
http://arxiv.org/pdf/2401.11246v1
The Good and The Bad: Exploring Privacy Issues in Retrieval-Augmented
  Generation (RAG)
http://arxiv.org/pdf/2402.16893v1
CLAPNQ: Cohesive Long-form Answers from Passages in Natural Questions
  for RAG systems
http://arxiv.org/pdf/2404.02103v1
Evaluation of Retrieval-Augmented Generation: A Survey
http://arxiv.org/pdf/2405.07437v1
FlashRAG: A Modular Toolkit for Efficient Retrieval-Augmented Generati

#Parsing and Chunking

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Directory setup
pdf_directory = '/content/drive/MyDrive/RAG_Papers'
output_directory= '/content/Parsed_Files'
zip_file= 'cleaned_texts.zip'
# Ensure the output directory exists


###**Parsing**
Parsing was done using llmparser**

In [ ]:
!pip install -U llama-index --upgrade -q
!pip install llama-parse -q

In [ ]:
!pip install llama-index-embeddings-huggingface -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 47.0 MB/s eta 0:00:00


In [ ]:
import os

# Set your API key (ensure this is done before initializing the parser)
os.environ['LLAMA_API_KEY'] = 'llx-78xMTtNuY8KS0UxiCo1jh8azXZlwWONmUE8eLZW77bNKNNdD'
print(os.getenv('LLAMA_API_KEY'))




llx-78xMTtNuY8KS0UxiCo1jh8azXZlwWONmUE8eLZW77bNKNNdD


In [ ]:
import nest_asyncio
nest_asyncio.apply()

from llama_parse import LlamaParse

parser = LlamaParse(
    api_key=os.getenv('LLAMA_API_KEY'),
    result_type="markdown",  # "markdown" and "text" are available
    num_workers=4,  # if multiple files passed, split in `num_workers` API calls
    verbose=True,
)


In [ ]:
# Get the first 45 PDF files
files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')][:30]
file_paths = [os.path.join(pdf_directory, file) for file in files]

# Parse the PDFs
documents = parser.load_data(file_paths)  # Load and parse documents

Parsing files: 100%|██████████| 30/30 [07:11<00:00, 14.37s/it]


###The documents returned represent the parsed content of the PDFs. The length of documents being 468 instead of 30 indicates that LlamaParse returns multiple segments or chunks per document rather than a single document. This chunking is essential for managing large documents more effectively and can help in scenarios like semantic chunking and retrieval augmentation.###

In [ ]:
print(len(documents))

468


**Data Cleansing**

In [ ]:
import zipfile
import unicodedata
import re
from google.colab import files as colab_files

def preprocess_text(text):
    """Replace multiple consecutive spaces with a single space."""
    text = re.sub(r'\nPage \d+\n', '', text)  # Example pattern for page numbers
    text = re.sub(r'\n(\w+\s+){1,4}\d+\n', '', text)  # Example pattern for headers/footers
    text = unicodedata.normalize('NFKC', text)  # Normalize unicode characters
    text = ' '.join(text.split())  # Remove excessive whitespace and newlines
    text = re.sub(r'\[\d+\]', '', text)  # Remove inline citations (e.g., [1])
    text = re.sub(r'\(\w+ et al\., \d{4}\)', '', text)  # Remove inline citations (e.g., (Smith et al., 2020))
    text = re.sub(r'References\n.*', '', text, flags=re.DOTALL)  # Remove references section
    return text

def save_text_to_file(filename, text, directory):
    """Save the cleaned text to a file."""
    with open(os.path.join(directory, filename.replace('.pdf', '.txt')), 'w', encoding='utf-8') as file:
        file.write(text)

# Process and save PDFs
for index, doc in enumerate(documents):
    cleaned_text = preprocess_text(doc.get_text())  # Use the appropriate method to get text
    save_text_to_file(f'document_{index}.txt', cleaned_text, output_directory)

def zip_files(directory, zip_filename):
    """Zip all files in a directory."""
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(directory):
            for file in files:
                zipf.write(os.path.join(root, file), arcname=file)

# Zip the processed files
zip_files(output_directory, zip_file)

print(f"Processed and zipped files are saved at {zip_file}")
files.download(zip_file)

Processed and zipped files are saved at cleaned_texts.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Semantic Chunking

In [ ]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Define the parameters for semantic chunking
buffer_size = 1
breakpoint_percentile_threshold = 95

# Function to chunk text using SemanticSplitterNodeParser
def chunk_text_with_semantic_splitter(document, buffer_size, breakpoint_percentile_threshold, embed_model):
    splitter = SemanticSplitterNodeParser(
        buffer_size=buffer_size,
        breakpoint_percentile_threshold=breakpoint_percentile_threshold,
        embed_model=embed_model
        # include_metadata=True,  # Include metadata
    )
    nodes = splitter.get_nodes_from_documents(document)
    return nodes

# Create semantic chunks
semantic_chunks = chunk_text_with_semantic_splitter(documents, buffer_size, breakpoint_percentile_threshold, embed_model)


In [ ]:
print("Number of semantic chunks created are:", len(semantic_chunks))
print(semantic_chunks[1].get_content)

Number of semantic chunks created are: 1232
<bound method TextNode.get_content of TextNode(id_='1593d631-cc17-4947-9306-4e9c8a8cdb90', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0a6be288-8f2e-435c-a681-83c89a3c0a3f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='27e5f0b23fe262224e232de1bc845d7e12d10b2c2245479d0718a07638756fc5'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='d769e7e9-bc9d-4ead-bcb0-8c0c97a0c12c', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='89ca1050aee3457b01f8abf90515a6156fa6c9a3777e9ca3269e6c090c8cffc6')}, text='A financial analyst might query, "Which company among Google, Apple, and Nvidia reported the largest profit margins in their third-quarter reports for 2023?" or inquire about a specific company’s performance over time, such as "How does Apple’s sales trend look over the past three years?" These queries requir

**Download the chunks**

In [ ]:
import json

# Function to save nodes with metadata
def save_node_with_metadata(node, index, directory):
    # Prepare the content to save
    content = {
        'text': node.get_content()
    }
    # Save as JSON file
    with open(os.path.join(directory, f'chunk_{index}.json'), 'w', encoding='utf-8') as file:
        json.dump(content, file, ensure_ascii=False, indent=4)

# Save all nodes
output_directory = '/content/Semantic_Chunks'  # Update with your path
os.makedirs(output_directory, exist_ok=True)

for index, node in enumerate(semantic_chunks):
    save_node_with_metadata(node, index, output_directory)


In [ ]:
import zipfile

# Function to zip the files
def zip_files(directory, zip_filename):
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(directory):
            for file in files:
                zipf.write(os.path.join(root, file), arcname=file)

# Define the zip filename
zip_filename = '/content/semantic_chunks.zip'  # Ensure it ends with .zip

# Zip the files
zip_files(output_directory, zip_filename)
colab_files.download(zip_filename)

print(f"Processed and zipped files are saved at {zip_filename}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Processed and zipped files are saved at /content/semantic_chunks.zip


#Using Qdrant to store the embeddings

In [ ]:
!pip install sentence-transformers -q

In [ ]:
!pip install transformers -U

  Using cached transformers-4.42.3-py3-none-any.whl (9.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastembed 0.2.7 requires huggingface-hub<0.21,>=0.20, but you have huggingface-hub 0.23.4 which is incompatible.
fastembed 0.2.7 requires tokenizers<0.16,>=0.15, but you have tokenizers 0.19.1 which is incompatible.


In [ ]:
%pip install llama-index-embeddings-huggingface -q
%pip install llama-index-embeddings-instructor -q

In [ ]:
!pip install llama-index -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 31.3 MB/s eta 0:00:00


In [ ]:
!pip install -U qdrant_client fastembed -q
# !pip install -U fastembed

In [ ]:
!pip install llama-index-embeddings-fastembed -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 15.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.41.2 requires huggingface-hub<1.0,>=0.23.0, but you have huggingface-hub 0.20.3 which is incompatible.
transformers 4.41.2 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is incompatible.


In [ ]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.2 which is incompatible.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
fastembed 0.2.7 requires huggingface-hub<0.21,>=0.20, but you have huggingface-hub 0.23.4 which is incompatible.
fastembed 0.2.7 req

In [ ]:
!pip install pyarrow -q

In [ ]:
!pip install llama-index-vector-stores-qdrant -q

In [ ]:
import logging
import sys
import os

import qdrant_client
from IPython.display import Markdown, display
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
# from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding



In [ ]:

# loads BAAI/bge-small-en
# embed_model = HuggingFaceEmbedding()

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from datasets import load_dataset

# Load a dataset from the Hugging Face Hub
# For example, the 'ag_news' dataset
dataset = load_dataset('Areeb-02/30rag_papers_qa_dataset', split = 'train')

# Inspect the dataset
print(dataset)

Dataset({
    features: ['instruction', 'system_prompt', '3B_q1', '3B_a1', '3B_q2', '3B_a2', '7B_q1', '7B_a1', '7B_q2', '7B_a2'],
    num_rows: 1010
})


In [ ]:
# prompt: getpass for qdrant cloud api

import getpass
qdrant_cloud_api_key = getpass.getpass(prompt='Qdrant Cloud API Key:')
os.environ['QDRANT_CLOUD_API_KEY'] = qdrant_cloud_api_key

Qdrant Cloud API Key:··········


In [ ]:
client = qdrant_client.QdrantClient(
    # you can use :memory: mode for fast and light-weight experiments,
    # it does not require to have Qdrant deployed anywhere
    # but requires qdrant-client >= 1.1.1
    # location=":memory:"
    # otherwise set Qdrant instance address with:
    url="https://be2766e1-cf84-4684-9d7d-7168e3cab398.us-east4-0.gcp.cloud.qdrant.io",
    # otherwise set Qdrant instance with host and port:
    # host="localhost",
    # port=6333
    # set API KEY for Qdrant Cloud
    api_key=qdrant_cloud_api_key,
)

In [ ]:
# documents are chunks from column
documents = dataset['instruction']

In [ ]:
vector_store = QdrantVectorStore(client=client, collection_name="rag_papers")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

AttributeError: 'str' object has no attribute 'get_doc_id'

In [ ]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()
document